## Imports

In [35]:
import numpy as np
import pandas as pd
import sys
from common import compute_rmse_fraction
import pickle
from create_matrix import *
from sklearn.metrics import mean_squared_error

## load predicted results

In [36]:
knn_p = pickle.load(open("predictions/knn.pkl",'r'))
knn_oracle_p = pickle.load(open("predictions/knn_oracle.pkl",'r'))

ml_p = pickle.load(open("predictions/metric_knn.pkl",'r'))
mf_p = pickle.load(open("predictions/mf.pkl",'r'))

tensor_parafac_p = pickle.load(open("predictions/tensor-parafac.pkl",'r'))
tensor_custom_p = pickle.load(open("predictions/tensor-custom.pkl",'r'))
tensor_custom_p_rel = pickle.load(open("predictions/tensor-custom-relative.pkl",'r'))

tensor_custom_parafac_static_p = pickle.load(open("predictions/tensor-custom-parafac-static-abs.pkl",'r'))
tensor_custom_parafac_static_rel_p = pickle.load(open("predictions/tensor-custom-parafac-static-rel.pkl",'r'))

tensor_custom_case_2_static_weather_rel_p = pickle.load(open("predictions/tensor-custom-2-static-weather-rel.pkl",'r'))
tensor_custom_case_2_static_weather_abs_p = pickle.load(open("predictions/tensor-custom-2-static-weather-abs.pkl",'r'))
tensor_custom_case_2_static_abs_p = pickle.load(open("predictions/tensor-custom-2-static-abs.pkl",'r'))
tensor_custom_case_2_static_rel_p = pickle.load(open("predictions/tensor-custom-2-static-rel.pkl",'r'))

In [25]:
def winorloss(pred_1, pred_2, appliance, region='Austin',year=2014):
    appliance_df = create_matrix_all_entries(region, year, appliance)
    
    if appliance == "hvac":
        start, stop = 5, 11
    else:
        start, stop = 1, 13
    pred_1 = pred_1.copy()
    pred_2 = pred_2.copy()
#     print "pred_1: ", pred_1.shape
#     print "pred_2: ", pred_1.shape
    
    pred_1.columns = [['%s_%d' % (appliance, month) for month in range(start, stop)]]
    pred_2.columns = [['%s_%d' % (appliance, month) for month in range(start, stop)]]
    
    gt_df = appliance_df[pred_1.columns].ix[pred_1.index]
    
    gt_df = gt_df.unstack().dropna()
    pred_1 = pred_1.unstack().dropna()
    pred_1 = pred_1.ix[gt_df.index]
    
    pred_2 = pred_2.unstack().dropna()
    pred_2 = pred_2.ix[gt_df.index]
    
#     print "pred_1: ", pred_1.shape
#     print "pred_2: ", pred_1.shape
#     print "gt_df: ", gt_df.shape

    result_1 = (pred_1 - gt_df)*(pred_1 - gt_df)/gt_df
    result_2 = (pred_2 - gt_df)*(pred_2 - gt_df)/gt_df
    result = result_1 - result_2
    
    pos = sum(x < 0 for x in result)
#     print pos
#     print gt_df.size
    return pos, gt_df.shape[0]-pos

# KNN

## RMSE

In [170]:
rmse_knn_results = {}
rmse_knn_latent = {}
for appliance in knn_p:
    rmse_knn_results[appliance] = {}
    rmse_knn_latent[appliance] = {}
    for feature in ['energy_static','energy']:
        rmse_knn_results[appliance][feature] = 1000
        least_error = 1000
        best_n = None
        for neighbours in range(1, 10):
            pred = knn_p[appliance][feature][neighbours]
            gt_f, pred_f, rms, e = compute_rmse_fraction(appliance, pred)
            if rms < least_error:
                least_error = rms
                best_n = neighbours
        rmse_knn_results[appliance][feature] = least_error
        rmse_knn_latent[appliance][feature] = best_n

In [171]:
pd.DataFrame(rmse_knn_latent)

,dw,fridge,hvac,mw,oven,wm
energy,9,7,8,9,9,9
energy_static,5,9,6,7,4,8


## Win/Loss

In [172]:
winloss_knn_results = {}
winloss_knn_latnet = {}
for appliance in knn_p:
    winloss_knn_results[appliance] = {}
    winloss_knn_latent[appliance] = {}
    for feature in ['energy_static','energy']:
        winloss_knn_results[appliance][feature] = np.zeros((9, 9))
#         least_error = 1000
#         best_n = None
        for neighbours_1 in range(1, 10):
            pred_1 = knn_p[appliance][feature][neighbours_1]
            for neighbours_2 in range(1, 10):
                if neighbours_1 == neighbours_2:
                    continue
                else:        
                    pred_2 = knn_p[appliance][feature][neighbours_2]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_knn_results[appliance][feature][neighbours_1-1][neighbours_2-1] = win
                    winloss_knn_results[appliance][feature][neighbours_2-1][neighbours_1-1] = loss
        factor_sum = winloss_knn_results[appliance][feature].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_knn_latent[appliance][feature] = best

In [173]:
pd.DataFrame(winloss_knn_latent)

,dw,fridge,hvac,mw,oven,wm
energy,9,9,4,9,9,9
energy_static,9,9,4,7,4,9


# KNN Oracle

In [174]:
knn_oracle = {}
for appliance in knn_oracle_p:
    a, b, knn_oracle[appliance], e = compute_rmse_fraction(appliance, knn_oracle_p[appliance])

# ML

## RMSE

In [175]:
rmse_ml_results = {}
rmse_ml_latent = {}
for appliance in ml_p:
    rmse_ml_results[appliance] = {}
    rmse_ml_latent[appliance] = {}
    for feature in ['energy_static','energy']:
        rmse_ml_results[appliance][feature] = 1000
        least_error = 1000
        best_n = None
        for neighbours in range(1, 10):
            pred = ml_p[appliance][feature][neighbours]
            gt_f, pred_f, rms, e = compute_rmse_fraction(appliance, pred)
            if rms < least_error:
                least_error = rms
                best_n = neighbours
        rmse_ml_results[appliance][feature] = least_error
        rmse_ml_latent[appliance][feature] = best_n

In [176]:
pd.DataFrame(rmse_ml_latent)

,dw,fridge,hvac,mw,oven,wm
energy,9,9,8,9,9,9
energy_static,9,8,8,6,9,9


## Win/Loss

In [177]:
winloss_ml_results = {}
winloss_ml_latent = {}
for appliance in ml_p:
    winloss_ml_results[appliance] = {}
    winloss_ml_latent[appliance] = {}
    for feature in ['energy_static','energy']:
        winloss_ml_results[appliance][feature] = np.zeros((9, 9))
#         least_error = 1000
#         best_n = None
        for neighbours_1 in range(1, 10):
            pred_1 = ml_p[appliance][feature][neighbours_1]
            for neighbours_2 in range(1, 10):
                if neighbours_1 == neighbours_2:
                    continue
                else:        
                    pred_2 = ml_p[appliance][feature][neighbours_2]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_ml_results[appliance][feature][neighbours_1-1][neighbours_2-1] = win
                    winloss_ml_results[appliance][feature][neighbours_2-1][neighbours_1-1] = loss
        factor_sum = winloss_ml_results[appliance][feature].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_ml_latent[appliance][feature] = best

In [178]:
pd.DataFrame(winloss_ml_latent)

,dw,fridge,hvac,mw,oven,wm
energy,9,9,4,9,9,9
energy_static,9,8,8,8,9,8


# Tensor Parafac

## RMSE

In [179]:
rmse_tensor_parafac_results = {}
rmse_tensor_parafac_latent = {}
for appliance in tensor_parafac_p:
#     rmse_tensor_parafac_results[appliance] = {}
#     rmse_tensor_parafac_latent[appliance] = {}
    least_error = 1000
    best_n = None
    for neighbours in range(1, 7):
        pred = tensor_parafac_p[appliance][neighbours]
        gt_f, pred_f, rms, e = compute_rmse_fraction(appliance, pred)
        if rms < least_error:
            least_error = rms
            best_n = neighbours
    rmse_tensor_parafac_results[appliance] = least_error
    rmse_tensor_parafac_latent[appliance] = best_n

In [180]:
pd.DataFrame([rmse_tensor_parafac_latent])

,dw,fridge,hvac,mw,oven,wm
0,1,1,2,1,2,1


## Win/Loss

In [181]:
winloss_tensor_parafac_results = {}
winloss_tensor_parafac_latent = {}
for appliance  in tensor_parafac_p:
    winloss_tensor_parafac_results[appliance] = np.zeros((6, 6))
#     winloss_tensor_parafac_latent[appliance] = np.zeros((6, 6))
    for neighbours_1 in range(1, 7):
        pred_1 = tensor_parafac_p[appliance][neighbours_1]
        for neighbours_2 in range(1, 7):
            if neighbours_1 == neighbours_2:
                continue
            else:
                pred_2 = tensor_parafac_p[appliance][neighbours_2]
                win, loss = winorloss(pred_1, pred_2, appliance)
                winloss_tensor_parafac_results[appliance][neighbours_1-1][neighbours_2-1] = win
                winloss_tensor_parafac_results[appliance][neighbours_2-1][neighbours_1-1] = loss
    factor_sum = winloss_tensor_parafac_results[appliance].sum(axis = 1)
    best = factor_sum.argmax() + 1
    winloss_tensor_parafac_latent[appliance] = best

In [182]:
pd.DataFrame([winloss_tensor_parafac_latent])

,dw,fridge,hvac,mw,oven,wm
0,3,1,2,1,2,1


# Tensor custom P

## RMSE

In [183]:
rmse_tf_custom_result = {}
rmse_tf_custom_latent = {}
for appliance in tensor_custom_p:
    rmse_tf_custom_result[appliance] = {}
    rmse_tf_custom_latent[appliance] = {}
    for case in range(1, 5):
        best_error = 1e6
        best_n = 0
        for a in range(1, 6):
            alpha, beta, error, e = compute_rmse_fraction(appliance,tensor_custom_p[appliance][case][a])
            if error < best_error:
                best_error = error
                best_n = a
        rmse_tf_custom_result[appliance][case] = best_error
        rmse_tf_custom_latent[appliance][case] = best_n

In [184]:
pd.DataFrame(rmse_tf_custom_latent)

,dw,fridge,hvac,mw,oven,wm
1,4,5,2,5,5,5
2,5,2,4,3,3,1
3,5,2,1,5,3,2
4,5,5,3,5,5,3


## Win/Loss

In [185]:
winloss_tf_custom_result = {}
winloss_tf_custom_latent = {}
for appliance in tensor_custom_p:
    winloss_tf_custom_result[appliance] = {}
    winloss_tf_custom_latent[appliance] = {}
    for case in range(1,5):
        winloss_tf_custom_result[appliance][case] = np.zeros((5, 5))
        for a in range(1,6):
            pred_1 = tensor_custom_p[appliance][case][a]
            for b in range(1,6):
                if a == b:
                    continue
                else:
                    pred_2 = tensor_custom_p[appliance][case][b]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_tf_custom_result[appliance][case][a-1][b-1] = win
                    winloss_tf_custom_result[appliance][case][b-1][a-1] = loss
        factor_sum = winloss_tf_custom_result[appliance][case].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_tf_custom_latent[appliance][case] = best
                

In [186]:
pd.DataFrame(winloss_tf_custom_latent)

,dw,fridge,hvac,mw,oven,wm
1,4,5,3,5,5,4
2,1,2,2,1,2,3
3,2,1,1,3,2,1
4,2,1,3,5,5,1


# Tensor Custom p, relative

## RMSE

In [187]:
rmse_tf_custom_rel_result = {}
rmse_tf_custom_rel_latent = {}
for appliance in tensor_custom_p_rel:
    rmse_tf_custom_rel_result[appliance]= {}
    rmse_tf_custom_rel_latent[appliance] = {}
    for case in range(1, 5):
        best_error = 1e6
        best_n = 0
        for a in range(1, 6):
            alpha, beta, error, e = compute_rmse_fraction(appliance,tensor_custom_p_rel[appliance][case][a])
            if error < best_error:
                best_error = error
                best_n = a
        rmse_tf_custom_rel_result[appliance][case] = best_error
        rmse_tf_custom_rel_latent[appliance][case] = best_n

In [188]:
pd.DataFrame(rmse_tf_custom_rel_latent)

,dw,fridge,hvac,mw,oven,wm
1,2,2,2,2,1,2
2,3,5,2,5,2,2
3,2,2,1,2,2,2
4,4,5,1,2,3,5


## Win/Loss

In [189]:
winloss_tf_custom_rel_result = {}
winloss_tf_custom_rel_latent = {}
for appliance in tensor_custom_p_rel:
    winloss_tf_custom_rel_result[appliance] = {}
    winloss_tf_custom_rel_latent[appliance] = {}
    for case in range(1,5):
        winloss_tf_custom_rel_result[appliance][case] = np.zeros((5, 5))
        for a in range(1,6):
            pred_1 = tensor_custom_p_rel[appliance][case][a]
            for b in range(1,6):
                if a == b:
                    continue
                else:
                    pred_2 = tensor_custom_p_rel[appliance][case][b]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_tf_custom_rel_result[appliance][case][a-1][b-1] = win
                    winloss_tf_custom_rel_result[appliance][case][b-1][a-1] = loss
        factor_sum = winloss_tf_custom_rel_result[appliance][case].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_tf_custom_rel_latent[appliance][case] = best
                

In [190]:
pd.DataFrame(winloss_tf_custom_rel_latent)

,dw,fridge,hvac,mw,oven,wm
1,2,2,2,1,1,2
2,4,2,2,1,2,1
3,2,2,1,2,2,1
4,5,3,4,5,2,5


# Tensor Custom Parafac with Static, absolute

## RMSE

In [191]:
rmse_tf_custom_parafac_static_result = {}
rmse_tf_custom_parafac_static_latent = {}
for appliance in tensor_custom_parafac_static_p:
    rmse_tf_custom_parafac_static_result[appliance]= {}
    rmse_tf_custom_parafac_static_latent[appliance] = {}
    for case in range(4, 5):
        best_error = 1e6
        best_n = 0
        for a in range(1, 6):
            alpha, beta, error, e = compute_rmse_fraction(appliance, tensor_custom_parafac_static_p[appliance][case][a])
            if error < best_error:
                best_error = error
#                 tf_custom_parafac_static[appliance] = best_error
                best_n = a
        rmse_tf_custom_parafac_static_result[appliance][case] = best_error
        rmse_tf_custom_parafac_static_latent[appliance][case] = best_n

In [195]:
pd.DataFrame(rmse_tf_custom_parafac_static_latent)

,dw,fridge,hvac,mw,oven,wm
4,4,4,3,2,2,4


## Win/Loss

In [196]:
winloss_tf_custom_parafac_static_result = {}
winloss_tf_custom_parafac_static_latent = {}
for appliance in tensor_custom_parafac_static_p:
    winloss_tf_custom_parafac_static_result[appliance] = {}
    winloss_tf_custom_parafac_static_latent[appliance] = {}
    for case in range(4, 5):
        winloss_tf_custom_parafac_static_result[appliance][case] = np.zeros((5, 5))
        for a in range(1,6):
            pred_1 = tensor_custom_parafac_static_p[appliance][case][a]
            for b in range(1,6):
                if a == b:
                    continue
                else:
                    pred_2 = tensor_custom_parafac_static_p[appliance][case][b]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_tf_custom_parafac_static_result[appliance][case][a-1][b-1] = win
                    winloss_tf_custom_parafac_static_result[appliance][case][b-1][a-1] = loss
        factor_sum = winloss_tf_custom_parafac_static_result[appliance][case].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_tf_custom_parafac_static_latent[appliance][case] = best
                

In [197]:
pd.DataFrame(winloss_tf_custom_parafac_static_latent)

,dw,fridge,hvac,mw,oven,wm
4,4,3,5,1,3,4


# Tensor Custom Parafac with Static, relative

In [198]:
rmse_tf_custom_parafac_static_rel_result = {}
rmse_tf_custom_parafac_static_rel_latent = {}
for appliance in tensor_custom_parafac_static_rel_p:
    rmse_tf_custom_parafac_static_rel_result[appliance]= {}
    rmse_tf_custom_parafac_static_rel_latent[appliance] = {}
    for case in range(4, 5):
        best_error = 1e6
        best_n = 0
        for a in range(1, 6):
            alpha, beta, error, e = compute_rmse_fraction(appliance, tensor_custom_parafac_static_rel_p[appliance][case][a])
            if error < best_error:
                best_error = error
                best_n = a
        rmse_tf_custom_parafac_static_rel_result[appliance][case] = best_error
        rmse_tf_custom_parafac_static_rel_latent[appliance][case] = best_n

In [199]:
pd.DataFrame(rmse_tf_custom_parafac_static_rel_latent)

,dw,fridge,hvac,mw,oven,wm
4,4,4,3,4,5,3


In [200]:
winloss_tf_custom_parafac_static_rel_result = {}
winloss_tf_custom_parafac_static_rel_latent = {}
for appliance in tensor_custom_parafac_static_rel_p:
    winloss_tf_custom_parafac_static_rel_result[appliance] = {}
    winloss_tf_custom_parafac_static_rel_latent[appliance] = {}
    for case in range(4, 5):
        winloss_tf_custom_parafac_static_rel_result[appliance][case] = np.zeros((5, 5))
        for a in range(1,6):
            pred_1 = tensor_custom_parafac_static_rel_p[appliance][case][a]
            for b in range(1,6):
                if a == b:
                    continue
                else:
                    pred_2 = tensor_custom_parafac_static_rel_p[appliance][case][b]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_tf_custom_parafac_static_rel_result[appliance][case][a-1][b-1] = win
                    winloss_tf_custom_parafac_static_rel_result[appliance][case][b-1][a-1] = loss
        factor_sum = winloss_tf_custom_parafac_static_rel_result[appliance][case].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_tf_custom_parafac_static_rel_latent[appliance][case] = best
                

In [201]:
pd.DataFrame(winloss_tf_custom_parafac_static_rel_latent)

,dw,fridge,hvac,mw,oven,wm
4,4,4,4,4,1,5


# Tensor Custom Case 2 with Static and Weather, absolute

In [204]:
rmse_tf_custom_2_static_weather_abs_result = {}
rmse_tf_custom_2_static_weather_abs_latent = {}
for appliance in tensor_custom_case_2_static_weather_abs_p:
    rmse_tf_custom_2_static_weather_abs_result[appliance]= {}
    rmse_tf_custom_2_static_weather_abs_latent[appliance] = {}
    for case in range(2, 3):
        best_error = 1e6
        best_n = 0
        for a in range(1, 6):
            alpha, beta, error, e = compute_rmse_fraction(appliance, tensor_custom_case_2_static_weather_abs_p[appliance][case][a])
            if error < best_error:
                best_error = error
                best_n = a
        rmse_tf_custom_2_static_weather_abs_result[appliance][case] = best_error
        rmse_tf_custom_2_static_weather_abs_latent[appliance][case] = best_n

In [205]:
pd.DataFrame(rmse_tf_custom_2_static_weather_abs_latent)

,dw,fridge,hvac,mw,oven,wm
2,2,4,3,2,3,2


In [211]:
winloss_tf_custom_2_static_weather_abs_result = {}
winloss_tf_custom_2_static_weather_abs_latent = {}
for appliance in tensor_custom_case_2_static_weather_abs_p:
    winloss_tf_custom_2_static_weather_abs_result[appliance] = {}
    winloss_tf_custom_2_static_weather_abs_latent[appliance] = {}
    for case in range(2, 3):
        winloss_tf_custom_2_static_weather_abs_result[appliance][case] = np.zeros((5, 5))
        for a in range(1,6):
            pred_1 = tensor_custom_case_2_static_weather_abs_p[appliance][case][a]
            for b in range(1,6):
                if a == b:
                    continue
                else:
                    pred_2 = tensor_custom_case_2_static_weather_abs_p[appliance][case][b]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_tf_custom_2_static_weather_abs_result[appliance][case][a-1][b-1] = win
                    winloss_tf_custom_2_static_weather_abs_result[appliance][case][b-1][a-1] = loss
        factor_sum = winloss_tf_custom_2_static_weather_abs_result[appliance][case].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_tf_custom_2_static_weather_abs_latent[appliance][case] = best


In [212]:
pd.DataFrame(winloss_tf_custom_2_static_weather_abs_latent)

,dw,fridge,hvac,mw,oven,wm
2,2,2,4,2,3,2


# Tensor Custom Case 2 with Static and Weather, relative

In [213]:
rmse_tf_custom_2_static_weather_rel_result = {}
rmse_tf_custom_2_static_weather_rel_latent = {}
for appliance in tensor_custom_case_2_static_weather_rel_p:
    rmse_tf_custom_2_static_weather_rel_result[appliance]= {}
    rmse_tf_custom_2_static_weather_rel_latent[appliance] = {}
    for case in range(2, 3):
        best_error = 1e6
        best_n = 0
        for a in range(1, 6):
            alpha, beta, error, e = compute_rmse_fraction(appliance, tensor_custom_case_2_static_weather_rel_p[appliance][case][a])
            if error < best_error:
                best_error = error
                best_n = a
        rmse_tf_custom_2_static_weather_rel_result[appliance][case] = best_error
        rmse_tf_custom_2_static_weather_rel_latent[appliance][case] = best_n

In [214]:
pd.DataFrame(rmse_tf_custom_2_static_weather_rel_latent)

,dw,fridge,hvac,mw,oven,wm
2,2,3,3,2,4,2


In [217]:
winloss_tf_custom_2_static_weather_rel_result = {}
winloss_tf_custom_2_static_weather_rel_latent = {}
for appliance in tensor_custom_case_2_static_weather_rel_p:
    winloss_tf_custom_2_static_weather_rel_result[appliance] = {}
    winloss_tf_custom_2_static_weather_rel_latent[appliance] = {}
    for case in range(2,3):
        winloss_tf_custom_2_static_weather_rel_result[appliance][case] = np.zeros((5, 5))
        for a in range(1,6):
            pred_1 = tensor_custom_case_2_static_weather_rel_p[appliance][case][a]
            for b in range(1,6):
                if a == b:
                    continue
                else:
                    pred_2 = tensor_custom_case_2_static_weather_rel_p[appliance][case][b]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_tf_custom_2_static_weather_rel_result[appliance][case][a-1][b-1] = win
                    winloss_tf_custom_2_static_weather_rel_result[appliance][case][b-1][a-1] = loss
        factor_sum = winloss_tf_custom_2_static_weather_rel_result[appliance][case].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_tf_custom_2_static_weather_rel_latent[appliance][case] = best
                

In [218]:
pd.DataFrame(winloss_tf_custom_2_static_weather_rel_latent)

,dw,fridge,hvac,mw,oven,wm
2,2,2,4,3,4,2


# Tensor Custom Case 2 with Static, absolute

In [219]:
rmse_tf_custom_2_static_abs_result = {}
rmse_tf_custom_2_static_abs_latent = {}
for appliance in tensor_custom_case_2_static_abs_p:
    rmse_tf_custom_2_static_abs_result[appliance]= {}
    rmse_tf_custom_2_static_abs_latent[appliance] = {}
    for case in range(2, 3):
        best_error = 1e6
        best_n = 0
        for a in range(1, 6):
            alpha, beta, error, e = compute_rmse_fraction(appliance, tensor_custom_case_2_static_abs_p[appliance][case][a])
            if error < best_error:
                best_error = error
                best_n = a
        rmse_tf_custom_2_static_abs_result[appliance][case] = best_error
        rmse_tf_custom_2_static_abs_latent[appliance][case] = best_n

In [220]:
pd.DataFrame(rmse_tf_custom_2_static_abs_latent)

,dw,fridge,hvac,mw,oven,wm
2,3,3,5,1,2,2


In [223]:
winloss_tf_custom_2_static_abs_result = {}
winloss_tf_custom_2_static_abs_latent = {}
for appliance in tensor_custom_case_2_static_abs_p:
    winloss_tf_custom_2_static_abs_result[appliance] = {}
    winloss_tf_custom_2_static_abs_latent[appliance] = {}
    for case in range(2, 3):
        winloss_tf_custom_2_static_abs_result[appliance][case] = np.zeros((5, 5))
        for a in range(1,6):
            pred_1 = tensor_custom_case_2_static_abs_p[appliance][case][a]
            for b in range(1,6):
                if a == b:
                    continue
                else:
                    pred_2 = tensor_custom_case_2_static_abs_p[appliance][case][b]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_tf_custom_2_static_abs_result[appliance][case][a-1][b-1] = win
                    winloss_tf_custom_2_static_abs_result[appliance][case][b-1][a-1] = loss
        factor_sum = winloss_tf_custom_2_static_abs_result[appliance][case].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_tf_custom_2_static_abs_latent[appliance][case] = best
                

In [224]:
pd.DataFrame(winloss_tf_custom_2_static_abs_latent)

,dw,fridge,hvac,mw,oven,wm
2,3,2,4,1,3,1


# Tensor Custom Case 2 with Static, relative

In [225]:
rmse_tf_custom_2_static_rel_result = {}
rmse_tf_custom_2_static_rel_latent = {}
for appliance in tensor_custom_case_2_static_rel_p:
    rmse_tf_custom_2_static_rel_result[appliance]= {}
    rmse_tf_custom_2_static_rel_latent[appliance] = {}
    for case in range(2, 3):
        best_error = 1e6
        best_n = 0
        for a in range(1, 6):
            alpha, beta, error, e = compute_rmse_fraction(appliance, tensor_custom_case_2_static_rel_p[appliance][case][a])
            if error < best_error:
                best_error = error
                best_n = a
        rmse_tf_custom_2_static_rel_result[appliance][case] = best_error
        rmse_tf_custom_2_static_rel_latent[appliance][case] = best_n

In [226]:
pd.DataFrame(rmse_tf_custom_2_static_rel_latent)

,dw,fridge,hvac,mw,oven,wm
2,3,5,3,2,3,2


In [229]:
winloss_tf_custom_2_static_rel_result = {}
winloss_tf_custom_2_static_rel_latent = {}
for appliance in tensor_custom_case_2_static_rel_p:
    winloss_tf_custom_2_static_rel_result[appliance] = {}
    winloss_tf_custom_2_static_rel_latent[appliance] = {}
    for case in range(2, 3):
        winloss_tf_custom_2_static_rel_result[appliance][case] = np.zeros((5, 5))
        for a in range(1,6):
            pred_1 = tensor_custom_case_2_static_rel_p[appliance][case][a]
            for b in range(1,6):
                if a == b:
                    continue
                else:
                    pred_2 = tensor_custom_case_2_static_rel_p[appliance][case][b]
                    win, loss = winorloss(pred_1, pred_2, appliance)
                    winloss_tf_custom_2_static_rel_result[appliance][case][a-1][b-1] = win
                    winloss_tf_custom_2_static_rel_result[appliance][case][b-1][a-1] = loss
        factor_sum = winloss_tf_custom_2_static_rel_result[appliance][case].sum(axis = 1)
        best = factor_sum.argmax() + 1
        winloss_tf_custom_2_static_rel_latent[appliance][case] = best
                

In [230]:
pd.DataFrame(winloss_tf_custom_2_static_rel_latent)

,dw,fridge,hvac,mw,oven,wm
2,2,2,3,1,2,4


# Matrix Factorization

In [ ]:
winloss_mf_results = {}
win_matrix = np.zeros((5,1))
for appliance in mf_p:
    winloss_mf_results[appliance] = {}
    for feature in ['energy_static','energy']:
        winloss_mf_results[appliance][feature] = {}
        for cost in ['relative','absolute']:
            winloss_mf_results[appliance][feature][cost] = np.zeros((5, 5))
#             print mf_results[appliance][feature][cost]
            for lat_1 in range(3, 8):
                for lat_2 in range(3, 8):
                    if lat_1 == lat_2:
                        continue
                    else:
                        pred_1 = mf_p[appliance][feature][cost][lat_1]
                        pred_2 = mf_p[appliance][feature][cost][lat_2]
                        win, loss = winorloss(pred_1, pred_2, appliance)
#                         print win, loss
                        winloss_mf_results[appliance][feature][cost][lat_1 - 3][lat_2 - 3] = win
                        winloss_mf_results[appliance][feature][cost][lat_2 - 3][lat_1 - 3] = loss
            print appliance, feature, cost
#             print mf_results[appliance][feature][cost]

In [ ]:
cost = 'relative'
feature = 'energy'

In [88]:
# win_matrix = np.zeros((5,1))
# for appliance in mf_p:
#     factor_sum = mf_results[appliance][feature][cost].sum(axis = 1).reshape(5,1)
#     win_matrix = win_matrix + factor_sum
#     print appliance, feature, cost
#     print win_matrix

In [46]:
appliance = 'fridge'
factor_sum =  winloss_mf_results[appliance][feature][cost].sum(axis = 1).reshape(5, 1)
whole_sum = winloss_mf_results[appliance][feature][cost].sum(axis=0)[0] + winloss_mf_results[appliance][feature][cost].sum(axis=1)[0]
all_results = np.concatenate((winloss_mf_results[appliance][feature][cost], factor_sum), axis=1)
all_results = np.concatenate((all_results, factor_sum/whole_sum), axis=1)
pd_index = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7"]
pd_column = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7", "sum", "ratio"]
print "Fridge  "
pd.DataFrame(data=all_results, index=pd_index, columns=pd_column)

Fridge  


,factor_3,factor_4,factor_5,factor_6,factor_7,sum,ratio
factor_3,0.0,274.0,225.0,286.0,226.0,1011.0,0.501488
factor_4,230.0,0.0,225.0,266.0,235.0,956.0,0.474206
factor_5,279.0,279.0,0.0,296.0,234.0,1088.0,0.539683
factor_6,218.0,238.0,208.0,0.0,226.0,890.0,0.441468
factor_7,278.0,269.0,270.0,278.0,0.0,1095.0,0.543155


In [30]:
appliance = 'hvac'
factor_sum =  winloss_mf_results[appliance][feature][cost].sum(axis = 1).reshape(5, 1)
all_results = np.concatenate((winloss_mf_results[appliance][feature][cost], factor_sum), axis=1)
pd_index = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7"]
pd_column = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7", "sum"]
print "hvac"
pd.DataFrame(data=all_results, index=pd_index, columns=pd_column)

hvac


,factor_3,factor_4,factor_5,factor_6,factor_7,sum
factor_3,0.0,296.0,199.0,314.0,273.0,1082.0
factor_4,238.0,0.0,197.0,328.0,280.0,1043.0
factor_5,335.0,337.0,0.0,349.0,338.0,1359.0
factor_6,220.0,206.0,185.0,0.0,232.0,843.0
factor_7,261.0,254.0,196.0,302.0,0.0,1013.0


In [31]:
appliance = 'wm'
factor_sum =  winloss_mf_results[appliance][feature][cost].sum(axis = 1).reshape(5, 1)
all_results = np.concatenate((winloss_mf_results[appliance][feature][cost], factor_sum), axis=1)
pd_index = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7"]
pd_column = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7", "sum"]
print "Washing Machine"
pd.DataFrame(data=all_results, index=pd_index, columns=pd_column)

Washing Machine


,factor_3,factor_4,factor_5,factor_6,factor_7,sum
factor_3,0.0,211.0,172.0,173.0,176.0,732.0
factor_4,185.0,0.0,167.0,137.0,169.0,658.0
factor_5,224.0,229.0,0.0,195.0,235.0,883.0
factor_6,223.0,259.0,201.0,0.0,240.0,923.0
factor_7,220.0,227.0,161.0,156.0,0.0,764.0


In [32]:
appliance = 'mw'
factor_sum =  winloss_mf_results[appliance][feature][cost].sum(axis = 1).reshape(5, 1)
all_results = np.concatenate((winloss_mf_results[appliance][feature][cost], factor_sum), axis=1)
pd_index = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7"]
pd_column = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7", "sum"]
print "Microwave"
pd.DataFrame(data=all_results, index=pd_index, columns=pd_column)

Microwave


,factor_3,factor_4,factor_5,factor_6,factor_7,sum
factor_3,0.0,162.0,147.0,206.0,190.0,705.0
factor_4,150.0,0.0,166.0,191.0,199.0,706.0
factor_5,165.0,146.0,0.0,188.0,172.0,671.0
factor_6,106.0,121.0,124.0,0.0,179.0,530.0
factor_7,122.0,113.0,140.0,133.0,0.0,508.0


In [33]:
appliance = 'oven'
factor_sum =  mf_results[appliance][feature][cost].sum(axis = 1).reshape(5, 1)
all_results = np.concatenate((mf_results[appliance][feature][cost], factor_sum), axis=1)
pd_index = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7"]
pd_column = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7", "sum"]
print "Oven"
pd.DataFrame(data=all_results, index=pd_index, columns=pd_column)

Oven


,factor_3,factor_4,factor_5,factor_6,factor_7,sum
factor_3,0.0,92.0,123.0,113.0,120.0,448.0
factor_4,136.0,0.0,128.0,120.0,137.0,521.0
factor_5,105.0,100.0,0.0,121.0,118.0,444.0
factor_6,115.0,108.0,107.0,0.0,116.0,446.0
factor_7,108.0,91.0,110.0,112.0,0.0,421.0


In [34]:
appliance = 'dw'

factor_sum =  mf_results[appliance][feature][cost].sum(axis = 1).reshape(5, 1)
all_results = np.concatenate((mf_results[appliance][feature][cost], factor_sum), axis=1)
pd_index = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7"]
pd_column = ["factor_3", "factor_4", "factor_5", "factor_6", "factor_7", "sum"]
print "Dish Washer"
pd.DataFrame(data=all_results, index=pd_index, columns=pd_column)

Dish Washer


,factor_3,factor_4,factor_5,factor_6,factor_7,sum
factor_3,0.0,143.0,147.0,132.0,167.0,589.0
factor_4,169.0,0.0,157.0,160.0,150.0,636.0
factor_5,165.0,155.0,0.0,153.0,163.0,636.0
factor_6,180.0,152.0,159.0,0.0,140.0,631.0
factor_7,145.0,162.0,149.0,172.0,0.0,628.0


## calculate the best factor for mf based on win/loss

In [74]:
winloss_mf_latent = {}
for appliance in mf_p:
    winloss_mf_latent[appliance] = {}
    for feature in ['energy_static', 'energy']:
        winloss_mf_latent[appliance][feature] = {}
        for cost in ['relative', 'absolute']:
            factor_sum = winloss_mf_results[appliance][feature][cost].sum(axis = 1)
#             best = factor_sum.index(max(factor_sum))
#             indices = np.where(a == a.max())
            best = factor_sum.argmax()
            winloss_mf_latent[appliance][feature][cost] = best + 3
print winloss_mf_latent['fridge']['energy']['relative']

7


In [77]:
winloss_mf_relative = {}
winloss_mf_abs = {}
for appliance in winloss_mf_results.keys():
    for cost in ['relative','absolute']:
        if cost=="absolute":
            winloss_mf_abs[appliance] =  pd.DataFrame(winloss_mf_latent[appliance]).ix[cost]
        else:
            winloss_mf_relative[appliance] =  pd.DataFrame(winloss_mf_latent[appliance]).ix[cost]

In [86]:
print "win/loss: mf_relative"
pd.DataFrame(winloss_mf_relative)


win/loss: mf_relative


,dw,fridge,hvac,mw,oven,wm
energy,4,7,5,4,4,6
energy_static,3,3,5,3,6,4


In [87]:
print "win/loss: absolute"
pd.DataFrame(winloss_mf_abs)

win/loss: absolute


,dw,fridge,hvac,mw,oven,wm
energy,7,4,7,3,4,3
energy_static,3,3,4,3,3,5


## calculate the best factor for mf based on RMSE

In [58]:
mf_results = {}
rmse_mf_latent = {}
for appliance in mf_p:
    mf_results[appliance] = {}
    rmse_mf_latent[appliance] = {}
    for feature in ['energy_static','energy']:
        mf_results[appliance][feature] = {}
        rmse_mf_latent[appliance][feature] = {}
        for cost in ['relative','absolute']:
            mf_results[appliance][feature][cost] = {}
            rmse_mf_latent[appliance][feature][cost] = {}
            least_error = 1000
            for lat in range(3, 8):
                
                pred = mf_p[appliance][feature][cost][lat]
                gt_f, pred_f, rms,e  = compute_rmse_fraction(appliance, pred)
                if rms < least_error:
                    least_error = rms
                    best_k = lat
#             print lat, cost, feature, appliance
#             print best_k, least_error
            mf_results[appliance][feature][cost] = least_error
            rmse_mf_latent[appliance][feature][cost] = best_k

5 13.9521429822
4 12.375774955
5 13.469415368
7 13.3840749751
3 0.421989697507
5 0.415035421109
6 0.432161887913
3 0.410138601918
3 4.13605389051
5 3.45110746712
5 4.24984004827
5 3.7090804557
3 2.04396191455
3 1.94103639731
3 1.99015085019
3 1.94335700744
7 1.28259276358
3 1.19365338472
6 1.38564200487
3 1.15585691192
3 0.91622326559
3 0.787991184931
7 1.36225438741
7 1.01248136816


In [59]:
mf_relative = {}
mf_abs = {}
for appliance in mf_results.keys():
    for cost in ['relative','absolute']:
        if cost=="absolute":
            mf_abs[appliance] =  pd.DataFrame(mf_results[appliance]).ix[cost]
        else:
            mf_relative[appliance] =  pd.DataFrame(mf_results[appliance]).ix[cost]

In [60]:
latent_relative = {}
latent_abs = {}
for appliance in best_latent.keys():
    for cost in ['relative', 'absolute']:
        if cost == "absolute":
            latent_abs[appliance] = pd.DataFrame(rmse_mf_latent[appliance]).ix[cost]
        else:
            latent_relative[appliance] = pd.DataFrame(rmse_mf_latent[appliance]).ix[cost]

In [61]:
pd.DataFrame(mf_relative)

,dw,fridge,hvac,mw,oven,wm
energy,1.362254,4.249840,13.469415,1.990151,1.385642,0.432162
energy_static,0.916223,4.136054,13.952143,2.043962,1.282593,0.421990


In [83]:
print "rmse: absolute"
pd.DataFrame(latent_abs)

rmse: absolute


,dw,fridge,hvac,mw,oven,wm
energy,7,5,7,3,3,3
energy_static,3,5,4,3,3,5


In [84]:
print "rmse: relative"
pd.DataFrame(latent_relative)

rmse: relative


,dw,fridge,hvac,mw,oven,wm
energy,7,5,5,3,6,6
energy_static,3,3,5,3,7,3
